Licensed under the Apache License, Version 2.0

In [ ]:
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

In [ ]:
def parse_example(serialized_example: bytes) -> dict[str, tf.Tensor]:
  # GOES-16 ABI band 8 to 16
  goes_bands = ('upper_vapor', 'mid_vapor', 'low_vapor', 'cloud_top', 'ozone',
                'data_10um', 'data_11um', 'data_12um', 'co2')
  feature_spec = {band: tf.io.FixedLenFeature([], tf.string) for band in goes_bands}
  feature_spec.update({
      'brightness_temperature_difference':  tf.io.FixedLenFeature([], tf.string),  # data_11um - data_12um
      'human_individual_masks': tf.io.FixedLenFeature([], tf.string),  # individual labeled masks
      'human_pixel_masks': tf.io.FixedLenFeature([], tf.string),  # majority vote from human_indvidual_masks      
      'n_times_before': tf.io.FixedLenFeature([], tf.int64),
      'n_times_after': tf.io.FixedLenFeature([], tf.int64),
      # Projection params
      'projection_wkt': tf.io.FixedLenFeature([], tf.string),
      'col_min': tf.io.FixedLenFeature([], tf.float32),
      'row_min': tf.io.FixedLenFeature([], tf.float32),
      'col_size': tf.io.FixedLenFeature([], tf.float32),
      'row_size': tf.io.FixedLenFeature([], tf.float32),
      # Timestamp
      'timestamp': tf.io.FixedLenFeature([], tf.int64),  # approximate timestamp
      'satellite_scan_starts': tf.io.VarLenFeature(tf.int64),  # timestamp from original file
  })
  features = tf.io.parse_single_example(serialized_example, feature_spec)
  for key in [*goes_bands, 'brightness_temperature_difference']:
    features[key] = tf.io.parse_tensor(features[key], tf.double)
  features['human_pixel_masks'] = tf.io.parse_tensor(features['human_pixel_masks'], tf.int32)
  features['human_individual_masks'] = tf.io.parse_tensor(features['human_individual_masks'], tf.int32)
  return features

## False color image
In order to view contrails in GOES, we use the "ash" color scheme. This color scheme was originally developed for viewing volcanic ash in the atmosphere but is also useful for viewing thin cirrus, including contrails. In this color scheme, contrails appear in the image as dark blue.

Note that we use a modified version of the ash color scheme here, developed by Kulik et al., which uses slightly different bands and bounds tuned for contrails.

References:

Ash Color Scheme (page 7): https://eumetrain.org/sites/default/files/2020-05/RGB_recipes.pdf

In [ ]:
_T11_BOUNDS = (243, 303)
_CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
_TDIFF_BOUNDS = (-4, 2)

def normalize_range(data, bounds):
  """Maps data to the range [0, 1]."""
  return (data - bounds[0]) / (bounds[1] - bounds[0])

def false_color_image(brightness_temperatures):
  """Generates ash false color image from GOES brightness temperatures."""
  r = normalize_range(brightness_temperatures['data_12um'] - brightness_temperatures['data_11um'], _TDIFF_BOUNDS)
  g = normalize_range(brightness_temperatures['data_11um'] - brightness_temperatures['cloud_top'], _CLOUD_TOP_TDIFF_BOUNDS)
  b = normalize_range(brightness_temperatures['data_11um'], _T11_BOUNDS)
  return np.clip(np.stack([r, g, b], axis=-1), 0, 1)


In [ ]:
dataset = tf.data.TFRecordDataset(tf.io.gfile.glob('gs://goes_contrails_dataset/20230419/tfrecords/train.tfrecords-00000-of-00100'))
dataset = dataset.map(parse_example)

In [ ]:
count = 0
for features in dataset.as_numpy_iterator():
  # Skip examples without contrails for visualization
  if features['human_pixel_masks'].sum() == 0:
    continue
  n_times_before = features['n_times_before']
  false_color_images = false_color_image(features)
  plt.figure(figsize=(12, 6))
  plt.subplot(1, 2, 1)
  plt.imshow(false_color_images[..., n_times_before, :])
  plt.subplot(1, 2, 2)
  plt.imshow(features['human_pixel_masks'].squeeze(-1))
  plt.show()
  count += 1
  if count >= 10:
    break